In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn import svm

In [0]:
class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('acc')>0.99:
      print("\n", "Ending Training as reached 99% accuracy!")
      self.model.stop_training = True

In [0]:
(training_data, training_labels), (test_data, test_labels) = keras.datasets.fashion_mnist.load_data()

training_data, test_data = training_data.reshape(60000, 28, 28, 1)/255.0, test_data.reshape(10000, 28, 28, 1)/255.0

model=keras.Sequential([
    keras.layers.Conv2D(64, (3,3), activation='relu', input_shape = (28, 28 ,1)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.summary()

4423680/4422102 [==============================] - 0s 0us/step


W0805 12:54:04.057530 140646579328896 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
flatten (Flatten)            (None, 43264)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               11075840  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 11,079,050
Trainable params: 11,079,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

fit_model = model.fit(training_data, training_labels, epochs = 20, callbacks=[myCallback()], validation_split=0.10)

model.evaluate(test_data, test_labels)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 113s 2ms/sample - loss: 0.3882 - acc: 0.8610 - val_loss: 0.3019 - val_acc: 0.8917
Epoch 2/20
54000/54000 [==============================] - 113s 2ms/sample - loss: 0.2540 - acc: 0.9057 - val_loss: 0.2806 - val_acc: 0.8982
Epoch 3/20
54000/54000 [==============================] - 113s 2ms/sample - loss: 0.1978 - acc: 0.9261 - val_loss: 0.2642 - val_acc: 0.9073
Epoch 4/20
54000/54000 [==============================] - 113s 2ms/sample - loss: 0.1537 - acc: 0.9418 - val_loss: 0.2820 - val_acc: 0.9082
Epoch 5/20
54000/54000 [==============================] - 114s 2ms/sample - loss: 0.1210 - acc: 0.9546 - val_loss: 0.3058 - val_acc: 0.9042
Epoch 6/20
54000/54000 [==============================] - 114s 2ms/sample - loss: 0.0939 - acc: 0.9649 - val_loss: 0.3235 - val_acc: 0.9072
Epoch 7/20
54000/54000 [==============================] - 114s 2ms/sample - loss: 0.0737 - acc: 0.9728 - val_lo

[0.5403376435302198, 0.9085]

In [0]:
from tensorflow.keras import Model
model_feat = Model(inputs = model.input, outputs = model.get_layer('dense').output)
feat_train = model_feat.predict(training_data)
feat_test = model_feat.predict(test_data)


In [0]:
print(feat_train.shape)

(60000, 256)


In [0]:
from sklearn.svm import SVC

svm = SVC(kernel = 'rbf', gamma = 1, random_state = 2, tol = 0.0001)

svm.fit(feat_train, training_labels)


In [0]:
svm.score(feat_train, training_labels)

In [0]:
svm.score(feat_test, test_labels)

In [0]:
import pandas as pd

history = pd.DataFrame(fit_model.history)
print(history)

In [0]:
plt.plot(fit_model.epoch, history.loss, '-r', marker='s', label = 'Loss')
plt.plot(fit_model.epoch, history.val_loss, '-b', marker='s', label = 'Val_loss')
plt.legend(loc='best')
plt.title('Loss')


In [0]:
plt.plot(fit_model.epoch, history.acc, '-r', marker='s', label = 'Loss')
plt.plot(fit_model.epoch, history.val_acc, '-b', marker='s', label = 'Val_loss')
plt.legend(loc='best')
plt.title('Accuracy')


In [0]:
pred = model.predict(test_data)

val = np.random.randint(0, 10000, [1, 15]).squeeze()

plt.figure(figsize=(2*2*3, 2*5))

for i in range(15):
  
  j = val[i]
  
  plt.subplot(5,6,2*i+1)
  bar = plt.bar(range(10), pred[j], color="#777777")
  
  if pred[j].argmax() == test_labels[j]:
    bar[pred[j].argmax()].set_color('blue')
  else:
    bar[pred[j].argmax()].set_color('red')
    
  plt.grid(False)
  plt.xticks(range(10), class_names, rotation = 'vertical')
  plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
  
    
  plt.subplot(5,6,2*i+2)
  plt.imshow(test_data[j].reshape(28,28))
  plt.title(class_names[test_labels[j]])
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
    
plt.tight_layout()
plt.show()